In [ ]:
library(tidyverse)
library(ggplot2)
library(openxlsx)
library(GGally)
library(reshape2)
library(plotly)

df <- readr::read_csv("/kaggle/input/coffee-quality-data-cqi/df_arabica_clean.csv")
df <- as.data.frame(df)
df <- df %>% 
  select("Country of Origin","Altitude","Aroma",
         "Flavor","Aftertaste","Acidity","Body","Balance",
         "Uniformity","Clean Cup","Sweetness")

In [ ]:
med <- round(apply(df[,3:8],2,mean),3)
med <- data.frame(med)
sd <- round(apply(df[,3:8],2,sd),3)
med <- cbind(med,Sd = sd)
colnames(med) <- c("Average","Dev")
med

In [ ]:
ggpairs(df,columns = 3:8,
        ggplot2::aes(color="blue"),
        upper = "blank")

In [ ]:
cor <- cor(df[,3:8])
m <- melt(cor)
colnames(m)=c("x","y","Cor")
m$Cor <- round(m$Cor,3)

p <- ggplot(data=m, aes(x,y,
                        fill=Cor)) + 
  geom_tile() + geom_text(aes(label=Cor),
                          color='white',size = 3) + 
  scale_fill_distiller(direction=1)  + 
  labs(x = "", y = "")

p 

**The 6 metrics have high correlations with each other, they are all direct relationships, that is, if a coffee receives a high score in any metric, flavor for example, it will also receive high scores in the other metrics.
Due to the high correlations, it is possible to use PCA and perform a more detailed analysis.**

In [ ]:
datos_pca <- prcomp(df[,3:8],scale=TRUE)
p <- ncol(df[,3:8])
eigen_datos <- datos_pca$sdev^2 
names(eigen_datos) <- paste("PC",1:p,sep="")
sumlambdas <- sum(eigen_datos)
propvar <- eigen_datos/sumlambdas*100
cumvar_datos <- cumsum(propvar)
matlambdas <- rbind(eigen_datos,propvar,cumvar_datos)
round(matlambdas,3)

**With 2 principal components it is enough, since with these 89.035% of the total variance are explained. We are going to give these new variables the names Z1 and Z2.**

In [ ]:
C.P <- round(datos_pca$rotation,3)
puntajes_pca <- data.frame(datos_pca$x)
PCA <- t(C.P[,1:2])
rownames(PCA) <- c("Z1","Z2")
PCA

**Z1, as shown, is negatively charged across all coffee metrics, therefore Z1 can be said to be an assessment of overall coffee quality, i.e. those coffees that score highest across all metrics will have a small value in Z1.
Instead, Z2 is a contrast between aroma, flavor, ftertaste and acidity, body, balance, for example, if a coffee has high values in aroma, flavor, ftertaste and low values in acidity, body, balance, it will have a small value. in Z2, otherwise, it will have a high value in Z2, if the coffee shows a balance in all metrics, it will have a medium value in Z2.**

In [ ]:
Z <- as.data.frame(datos_pca$x)
Sum <- apply(df[,3:8], 1,sum)
p <- ggplot(data = Z) + 
  geom_point(aes(x = PC1,y = PC2,color = Sum),
             size = 2) + 
  scale_color_gradient(low = "blue", high = "red") + 
  labs(x = "Z1", y = "Z2", color = "Sum")
p

**The scatter diagram shows the aforementioned, since those coffees with the sum of the highest ratings received are the coffees that appear furthest to the left, those points that are at the top of the graph are coffees that have high acidity scores, body and balance.**

## Clustering

**The grouping was done by the ward method, using the 6 original variables.**

In [ ]:
destand = apply(df[,3:8],2,function(x) (x-mean(x))/sd(x))
res = hclust(dist(destand),method= "ward.D2")  
plot(res,hang=-1,main="",ylab="Distancia Euclidiana")

**The clustering was done with the original data, the principal components created above can be very useful to describe each of the clusters.**

In [ ]:
Grupo <- cutree(res,k=2)
ggplot(data = Z) + 
  geom_point(aes(x = PC1,y = PC2,color = as.factor(Grupo)),
             size = 2) +
  labs(x = "Z1", y = "Z2", color = "Group")


**As previously stated, those coffees that have small values in Z1 are because they received high scores in all the evaluations. Having said this, it can be seen that group 1 is made up precisely of those coffees that have received the highest scores in all the evaluations. evaluations, so it could be said that they are the highest quality coffees.**

In [ ]:
df2 <- cbind(df[,3:8],Grupo)
g1 <- df2 %>% filter(Grupo==1)
data.frame(Promedio = apply(g1, 2,mean))

In [ ]:
g2 <- df2 %>% filter(Grupo==2)
data.frame(Promedio = apply(g2, 2,mean))